In [8]:
from pathlib import Path
import pandas as pd
import os
import os.path as osp
from datetime import datetime, time, timedelta

In [9]:
person_id = 'u1_extra'
query_path = Path.cwd().parent / 'data' / person_id / 'puzzle_query'
queries = [d for d in sorted(os.listdir(str(query_path))) if osp.isdir(str(query_path / d)) and not d.startswith('.')]
print(queries)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010']


In [10]:
def time_in_range(t):
    result = 0
    if time(4, 0, 0) <= t <= time(11, 59, 0):
        result = 1
    elif time(12, 0, 0) <= t <= time(16, 59, 0):
        result = 2
    elif time(17, 0, 0) <= t <= time(22, 59, 0):
        result = 3
    elif time(23, 0, 0) <= t or t >= time(3, 59, 0):
        result = 4
    return result

In [11]:
combine_gtdf = []
for query in queries:
    print('Processing query {}'.format(query))
    query_fold_path = query_path / query
    map_filepath = query_fold_path / 'map_df.csv'
    map_df = pd.read_csv(map_filepath)
    ids = list(map(lambda x: '{:%Y:%m:%d_:%H:%M}_{}'.format(
            datetime.strptime(x[1][3:16], '%Y%m%d_%H%M') + timedelta(hours=1), x[1][17:])
               .replace(':',''), map_df.values))
    times = list(map(lambda x: time_in_range(datetime.strptime(x[1][12:16], '%H%M').time()), map_df.values))
    query_id = [query] * len(times)
    map_df.insert(3, 'ids', ids)
    map_df.insert(4, 'times', times)
    map_df.sort_values('ids', axis=0, inplace=True)
    orders = [i+1 for i in range(len(map_df.index))]
    map_df.insert(5, 'orders', orders)
    map_df.sort_values('image_name', axis=0, inplace=True)
    map_df.drop(['image_id', 'image_path', 'ids'], axis=1, inplace=True)
    map_df = map_df[['image_name', 'orders', 'times']]
    map_df.insert(0, 'query_id', query_id)
#     map_df.to_csv(str(query_path / '{}.csv'.format(query)), index=False, header=False, na_rep='NULL')
    combine_gtdf.append(map_df)
combine_gtdf = pd.concat(combine_gtdf)
combine_gtdf.to_csv(str(query_path / 'puzzle_test_gt.csv'), index=False, header=False, na_rep='NULL')

Processing query 001
Processing query 002
Processing query 003
Processing query 004
Processing query 005
Processing query 006
Processing query 007
Processing query 008
Processing query 009
Processing query 010
